In [20]:
import torch
from PIL import Image, ImageDraw, ImageFont
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import sys
import os

# Add the YOLOv7 directory to the system path
sys.path.insert(0, r'D:\Kuliah Semester 6\Jurnal\Materi\Modul1\yolo7-cpu')

# Import YOLOv7 specific modules
from models.experimental import attempt_load
from utils.general import non_max_suppression, scale_coords
from utils.datasets import letterbox

# Define your class dictionary
dic = {0: 'Buras', 1: 'Sop Konro', 2: 'Kapurung', 3: 'Dangkot', 4: 'Gogos', 5: 'Sokko'}
label_to_class = {v: k for k, v in dic.items()}

# Function to preprocess image for classification models
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img) / 255.0
    img = np.expand_dims(img, axis=0)
    return img

# Function to perform object detection with YOLOv7
def detect_objects_yolov7(img_path):
    # Load YOLOv7 model
    weights_path = r"D:\Kuliah Semester 6\Jurnal\Materi\Modul1\yolo7-cpu\runs\train\yolov7-Makanan12\weights\best.pt"
    model = attempt_load(weights_path, map_location='cpu')

    # Load image
    img = Image.open(img_path)
    img = np.array(img)  # Convert to numpy array
    img = letterbox(img, new_shape=640)[0]  # Resize image

    # Convert to numpy array and normalize
    img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB
    img = np.ascontiguousarray(img)
    img = torch.from_numpy(img).float()
    img /= 255.0  # Normalize to [0, 1]
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    # Inference
    pred = model(img, augment=False)[0]
    pred = non_max_suppression(pred, 0.25, 0.45, classes=None, agnostic=False)

    detected_objects = []
    for det in pred:  # detections per image
        if len(det):
            # Rescale boxes from img_size to im0 size
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img.shape).round()
            for *xyxy, conf, cls in reversed(det):
                label = int(cls)
                bbox = (int(xyxy[0]), int(xyxy[1]), int(xyxy[2]), int(xyxy[3]))
                detected_objects.append({'label': label, 'bbox': bbox})

    return detected_objects

def ensemble_predict(img_path, mobilenetv2_model, xception_model, efficientnetb0_model):
    detected_objects = detect_objects_yolov7(img_path)

    if detected_objects:
        bbox = detected_objects[0]['bbox']
        cropped_img = Image.open(img_path).crop(bbox)
        cropped_img = cropped_img.resize((224, 224))  # Resize to match model input size
        processed_img = np.array(cropped_img) / 255.0  # Normalize image
        processed_img = np.expand_dims(processed_img, axis=0)

        mobilenetv2_pred = mobilenetv2_model.predict(processed_img)
        xception_pred = xception_model.predict(processed_img)
        efficientnetb0_pred = efficientnetb0_model.predict(processed_img)

        final_pred = (mobilenetv2_pred + xception_pred + efficientnetb0_pred) / 3.0
        return np.argmax(final_pred, axis=-1), detected_objects
    else:
        print("No objects detected in the image:", img_path)
        return None, None

# Input models
mobilenetv2_model = load_model(r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Codingan\model\mobilenetv2_Percobaan 4(2).h5")
xception_model = load_model(r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Codingan\Percobaan4_Xception.h5")
efficientnetb0_model = load_model(r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Codingan\Percobaan3(1)_EfficientNetB0.h5")

# Compile models to avoid warnings
mobilenetv2_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
xception_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
efficientnetb0_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Evaluate model
def evaluate_model(test_images, test_labels):
    true_labels = []
    pred_labels = []

    for img_path, true_label in zip(test_images, test_labels):
        pred_label, detected_objects = ensemble_predict(img_path, mobilenetv2_model, xception_model, efficientnetb0_model)
        if pred_label is not None and detected_objects is not None:
            true_labels.append(true_label)
            pred_labels.append(pred_label[0])
            
            # Display detected objects
            img = Image.open(img_path)
            draw = ImageDraw.Draw(img)
            
            # Load a font with a specified size
            font_size = 50
            font = ImageFont.truetype("arial.ttf", font_size)

            for obj in detected_objects:
                label = obj['label']
                bbox = obj['bbox']
                draw.rectangle(bbox, outline='red', width=2)
                
                # Calculate text size
                text = dic[label]
                text_bbox = draw.textbbox((bbox[0], bbox[1]), text, font=font)
                
                # Define text background box
                text_background = [text_bbox[0], text_bbox[1], text_bbox[2], text_bbox[3]]
                draw.rectangle(text_background, fill='red')
                
                # Draw text on top of the background
                draw.text((bbox[0], bbox[1]), text, fill='white', font=font)

            img.show()  # Show annotated image

    if true_labels and pred_labels:
        print("Classification Report:")
        print(classification_report(true_labels, pred_labels, target_names=list(dic.values()), labels=list(dic.keys())))
        print("Accuracy:", accuracy_score(true_labels, pred_labels))
        print("Precision:", precision_score(true_labels, pred_labels, average='weighted'))
        print("Recall:", recall_score(true_labels, pred_labels, average='weighted'))
        print("F1 Score:", f1_score(true_labels, pred_labels, average='weighted'))
    else:
        print("No predictions were made.")

# Example usage
test_images = [
    r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Dataset\Buras\BURAS_3.jpg",
    r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Split_dataset\test\Sop Konro\SOP KONRO_536.jpg",
    r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Split_dataset\test\Kapurung\KAPURUNG_1089.jpg",
    r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Split_dataset\test\Dangkot\DANGKOT_32.jpg",
    r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Dataset\Gogos\GOGOS_4.jpg",
    r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Dataset\Sokko\SOKKO_58.jpg",
]
test_labels = [
    0,
    1,
    2,
    3,
    4,
    5
]

evaluate_model(test_images, test_labels)


Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
No objects detected in the image: D:\Kuliah Semester 6\Jurnal\Materi\CNN\Split_dataset\test\Sop Konro\SOP KONRO_536.jpg
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
No objects detected in the image: D:\Kuliah Semester 6\Jurnal\Materi\CNN\Split_dataset\test\Kapurung\KAPURUNG_1089.jpg
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
No objects detected in the image: D:\Kuliah Semester 6\Jurnal\Materi\CNN\Split_dataset\test\Dangkot\DANGKOT_32.jpg
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetec

c:\Program Files\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Program Files\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Program Files\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Program Files\Python311\Lib\site